# Retrieval Notebook
### Imports

In [ ]:
import numpy as np
import re
import heapq
import pandas as pd
import time
import string
import pickle
import nltk
import random
from nltk.probability import FreqDist
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import matplotlib
import pylab as pl
%matplotlib inline

## Loading pickle variables (Language model, lyrics database)
### defined in Preprocessing Notebook
##### Download data from https://www.dropbox.com/s/h94ebpetwmw45sp/Data.zip?dl=0
Loading will take ~1 minute

In [ ]:
print("Loading Language models + relevant data")
start = time.time()
print('-----------------------------------------------------')
with open('Data/inverted_terms', 'rb') as f:
    inverted_terms = pickle.load(f)
print('loaded: inverted_terms-------------------------------')

with open('Data/inverted_bigrams', 'rb') as f:
    inverted_bigrams = pickle.load(f)
print('=====loaded: inverted_bigrams------------------------')

with open('Data/song_total_terms', 'rb') as f:
    song_total_terms = pickle.load(f)
print('==========loaded: song_total_terms-------------------')

with open('Data/song_total_bigrams', 'rb') as f:
    song_total_bigrams = pickle.load(f)
print('===============loaded: song_total_bigrams------------')

with open('Data/lyrics_original', 'rb') as f:
    lyrics_original = pickle.load(f) #lyrics with capitalisation / punctuation
print('======================loaded: lyrics_original--------')

with open('Data/tokenized_lyrics', 'rb') as f:
    tokenized_lyrics = pickle.load(f) #stemmed, tokenized lyrics (list of lists of terms)
print('=============================loaded: tokenized_lyrics')
print('=====================================================')

collection_N = sum([e[0] for e in inverted_terms.values()])
bigram_collection_N = sum([e[0] for e in inverted_bigrams.values()])

end = time.time()
print("All data loaded in {:0>2d}:{:0>2.0f}".format(int((end-start)//60), (end-start)%60))

## Retrieval models

In [ ]:
def retrieve_1(query, max_heap_size=100, smoothing=.85):
    '''
    base: inverted unigram language model
    Retrieves 'max_heap_size' indices of the lyrics in the lyrics DataFrame in decreasing order of 
    relevance to the query
    uses an inverted unigram language model defined in the preprocessing notebook 
    applies smoothing with smoothing factor defined by 'smoothing'
    uses only unigrams
    '''
    stemmer = PorterStemmer() #stemmer
    #max_heap is a list with (score, index) tuples, where the index is the index of the song in the dataframe
    max_heap = list(zip(np.repeat(0, max_heap_size), np.repeat(-1, max_heap_size))) #initialise max heap with (0, -1)
    query = re.sub('\W', ' ', query) #strip query from punctuation
    terms = [stemmer.stem(t) for t in word_tokenize(query.lower())] #stemmed, tokenized query terms
    
    scores = {} #{song index : score}
    for term in terms: #sum over query terms:
        if term in inverted_terms: #if term in the collection:
            collection_count = inverted_terms[term][0]
            for key in inverted_terms[term][1]: #for each document in which term occurs:
                #add term score (proportional to TF, inversely proportional to IDF) to total score of document
                score = np.log(1+ ((1-smoothing)*inverted_terms[term][1][key]/song_total_terms[key])\
                                                /((smoothing*collection_count)/collection_N))
                if key in scores:
                    scores[key] += score
                else:
                    scores[key] = score
    for key in scores:
        scores[key] += np.log(song_total_terms[key])/np.log(collection_N) #length prior
        if scores[key] > min(max_heap)[0]: #update max heap
            heapq.heappop(max_heap)
            heapq.heappush(max_heap, (scores[key],key))
    
    return [x[1] for x in sorted(max_heap, reverse=True)] #return indices of songs in decreasing order of relevance

In [ ]:
def retrieve_2(query, max_heap_size=100, smoothing=.85): 
    '''
    base: bigram language model with inverted file index
    same as retrieve_1 but uses bigrams
    '''
    stemmer = PorterStemmer() #stemmer
    #max_heap is a list with (score, index) tuples, where the index is the index of the song in the dataframe
    max_heap = list(zip(np.repeat(0, max_heap_size), np.repeat(-1, max_heap_size))) #initialise max heap with (0, -1)
    query = re.sub('\W', ' ', query) #strip query from punctuation
    terms = [stemmer.stem(t) for t in word_tokenize(query.lower())] #stemmed, tokenized query terms
    query_bigrams = list(nltk.bigrams(terms))
    
    scores = {} #{song index : score}
    for bigram in query_bigrams: #sum over query bigrams:
        if bigram in inverted_bigrams: #if bigram in the collection:
            collection_count = inverted_bigrams[bigram][0]
            for key in inverted_bigrams[bigram][1]: #for each document in which bigram occurs:
                #add bigram score (proportional to TF, inversely proportional to IDF) to total score of document
                score = np.log(1+ ((1-smoothing)*inverted_bigrams[bigram][1][key]/song_total_bigrams[key])\
                                                /((smoothing*collection_count)/bigram_collection_N))
                if key in scores:
                    scores[key] += score
                else:
                    scores[key] = score
    for key in scores:
        scores[key] += np.log(song_total_bigrams[key])/np.log(bigram_collection_N) #length prior
        if scores[key] > min(max_heap)[0]: #update max heap
            heapq.heappop(max_heap)
            heapq.heappush(max_heap, (scores[key],key))
            
    return [x[1] for x in sorted(max_heap, reverse=True)] #return indices of songs in decreasing order of relevance

### Rerank method

In [ ]:
def rerank_ngrams(indices, query, n=2): 
    '''
    rerank indices on ngram presence
    defines ngrams on the fly using nltk.ngrams with n = n
    this is rather quick since we have the list with terms for each song (tokenized_lyrics) and we only need to rerank
    max_heap_size indices (from retrieval method)
    '''
    stemmer = PorterStemmer()
    query = re.sub('\W', ' ', query) #strip query from punctuation
    terms = [stemmer.stem(t) for t in word_tokenize(query.lower())] #stemmed, tokenized query terms
    query_ngrams = list(nltk.ngrams(terms, n)) #all ngrams generated from query
    
    ngram_counts = [] #for each index in indices, (-) the number of query ngrams that occur in those lyrics
    for index in indices:
        if index == -1: # empty result
            ngram_counts.append(1) #this is worse than 0
            break
        #list of lists of ngrams for each lyrics in the top results
        all_ngrams = list(nltk.ngrams(tokenized_lyrics[index], n)) #all ngrams for lyrics at index
        count = 0
        for ngram in query_ngrams: #for every ngram from the query
            if ngram in all_ngrams: #if the nrgram is present in the lyrics: improve score
                count -= 1 #minus for sorting to remain order within the same counts (reverse sort reverses this)

        ngram_counts.append(count)
    
    #sort indices based on ngram_counts
    new_ranking = [index for _,index in sorted(zip(ngram_counts,indices), key=lambda x:x[0])]
    return new_ranking #indices from the lyrics dataframe in decreasing order of relevance

## Query testing

### proxy query

In [ ]:
'''
Get a query proxy: a string of k consecutive words extracted randomly from a random song
these queries are used to test the models and tweak hyperparameters
'''
def query_proxy(k=6):
    words=[]
    while(len(words)<k): #make sure the song has at least k words
        sample = lyrics_original.sample()
        words = sample['lyrics'].iloc[0].split() #split words

    r = random.randint(0,len(words)-k) #random location in the song lyrics
    query = ' '.join(words[r:r+k]) #slice k words from lyrics

    print('Query: ', query)
    print("Index: {}\nSong: {}\nArtist: {}".format(sample['index'].iloc[0], sample['song'].iloc[0],\
                                                   sample['artist'].iloc[0]))
    return query

### Get the ranked results from a query for all models 
#### results = indices of songs in the DataFrame
models: <br>
unigram model <br>
unigram model + bigram rerank<br>
bigram model<br>
bigram model + trigram rerank<br>

In [ ]:
##########################################################################
#################### DEFINE QUERY HERE OR GET A PROXY ####################
##########################################################################
query = "All I wanna hear her say is are you mine"
#query = query_proxy()
##########################################################################

max_heap_size = 100

print("\nGetting results...")
#unigram model
start = time.time()
indices_uni = retrieve_1(query, max_heap_size) #unigram model
uni_time = time.time()-start
print("time for unigram model results: \t\t{} seconds".format(uni_time))

#unigram model + bigram rerank
start = time.time()
indices_uni_bi = rerank_ngrams(indices_uni, query, 2) #rerank indices on bigrams
uni_bi_time = time.time()-start
print("time to rerank unigram results on bigrams: \t{} seconds".format(uni_bi_time))

#bigram model
start = time.time()
indices_bi = retrieve_2(query, max_heap_size) #bigram model
bi_time = time.time()-start
print("time for bigram model results: \t\t\t{} seconds".format(bi_time))

#bigram model + trigram rerank
start = time.time()
indices_bi_tri = rerank_ngrams(indices_bi, query, 3) #rerank indices on trigrams
bi_tri_time = time.time()-start
print("time to rerank bigram results on trigrams: \t{} seconds".format(bi_tri_time))

indices = [indices_uni, indices_uni_bi, indices_bi, indices_bi_tri]

### Print result page for each model (top 'nr_results_to_show')

In [ ]:
nr_results_to_show = 20

names = ['\033[1mmixed model + trigram rerank\033[0m', '\033[1munigram model\033[0m', \
         '\033[1munigram model + bigram rerank\033[0m', '\033[1mbigram model\033[0m',\
         '\033[1mbigram model + trigram rerank\033[0m']
i = 0
for result_page in indices:
    print(names[i])
    #print(result_page)
    i+=1
    rank=1
    for index in result_page[:min(nr_results_to_show, len(result_page))]:
        if(index==-1): #no more songs indexed
            break
        print("Rank: {}  Song: {}\tArtist: {}\tIndex: {}".format(rank, lyrics_original['song'].iloc[index], \
                                                      lyrics_original['artist'].iloc[index], index))
        rank+=1
    print("\n")

### Timing studies

In [ ]:
#previous timing study results (MacBook Pro 2018 8GB ram T2 Intel Core i5)

times_unigram = [0.6614131117, 0.9135156918, 1.36761508, 1.556321063, 1.612079, 1.63472198, 1.936166196, 2.028390484, \
2.588905816, 2.428667722, 2.679294305, 2.637917728, 2.530018735]
times_bigram = [0.07451394796, 0.1133789158, 0.1427968192, 0.1722764993, 0.265690732, 0.2704293275, 0.3036595631, \
0.371507051, 0.3504348707, 0.4463950658, 0.4693552208, 0.4954117918]

In [ ]:
pl.figure(figsize=(10, 6))
matplotlib.rc('xtick', labelsize=17) 
matplotlib.rc('ytick', labelsize=17) 
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)
plt.plot(range(1,14), times_unigram)
plt.plot(range(2,14), times_bigram)
plt.xlabel("nr of query terms")
plt.ylabel("seconds until result")
plt.legend(("unigram model*","bigram model**"))
plt.xscale('linear')
plt.xticks(range(1,14));

In [ ]:
if input("Are you sure you want to do the timing study again?").lower() == 'yes':
    times_unigram = []
    for nr_query_terms in range (1,14):
        print("queries with {} term(s)".format(nr_query_terms))
        seconds = []
        for iterations in range(50):
            query = query_proxy(nr_query_terms)
            start = time.time()
            x = retrieve_invert_uni(query, 100)
            seconds.append(time.time()-start)
        average = np.mean(seconds)
        print("average time: {}".format(average))
        times_unigram.append(average)

In [ ]:
if input("Are you sure you want to do the timing study again?").lower() == 'yes':
    times_bigram = []
    for nr_query_terms in range (2,14):
        print("queries with {} term(s)".format(nr_query_terms))
        seconds = []
        for iterations in range(100):
            query = query_proxy(nr_query_terms)
            start = time.time()
            x = retrieve_invert_bi(query, 100)
            seconds.append(time.time()-start)
        average = np.mean(seconds)
        print("average time: {}".format(average))
        times_bigram.append(average)